In [ ]:
from models_neural.quote_attribution.classification_models import SourceQA

In [1]:
import json
import attrdict
from models_neural.src.config_helper import TransformersConfig
import pandas as pd 

! aws s3 cp s3://aspangher/transformer-pretrained-models/roberta-base-expanded-embeddings.zip .  --endpoint http://s3.dev.obdc.bcs.bloomberg.com
        
import zipfile
with zipfile.ZipFile('roberta-base-expanded-embeddings.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [57]:
data_fn = 'our-annotated-data__stage-2.tsv'
data_df = pd.read_csv(data_fn, sep='\t', header=None)
model_path = 'trial-Stage 2: Quote Attribution. Our dataset only.__epoch=05-perplexity=0.00.ckpt'

In [51]:
! aws s3 cp s3://aspangher/source-exploration/./config-Stage\ 2\:\ Quote\ Attribution.\ Our\ dataset\ only..json  . --endpoint http://s3.dev.obdc.bcs.bloomberg.com

download: s3://aspangher/source-exploration/./config-Stage 2: Quote Attribution. Our dataset only..json to ./config-Stage 2: Quote Attribution. Our dataset only..json


In [54]:
config_out = 'config-%s.json' % "Stage 2: Quote Attribution. Our dataset only."
with open(config_out) as f:
    config_dict = json.load(f)

config_dict = attrdict.AttrDict(config_dict)
config_dict.pretrained_files_s3 = config.pretrained_model_path
config_dict.model_type = 'roberta'

In [55]:
model = SourceQA.load_from_checkpoint(model_path, config=config_dict)

In [56]:
from models_neural.quote_attribution.utils_dataset import SourceQADataModule

In [62]:
! aws s3 cp --recursive s3://aspangher/spacy/en_core_web_lg/ en_core_web_lg/   --endpoint http://s3.dev.obdc.bcs.bloomberg.com

'./roberta-base-expanded-embeddings'

In [71]:
data_model = SourceQADataModule(
    config=config_dict,
    data_fp=data_fn,
    model_type='roberta',
    max_length_seq=2048,
    pretrained_model_path=config_dict.pretrained_model_path,
    batch_size=1,
    spacy_path='en_core_web_lg'
)

In [72]:
data_model.prepare_data()

In [73]:
data_model.setup()

100%|██████████| 335/335 [02:15<00:00,  2.48it/s]


In [89]:
dataloader = data_model.val_dataloader()

In [92]:
dataloader.batch_size

1

In [ ]:
for sample in dataloader:
    sample

In [103]:
with torch.no_grad():
    output = model.forward(input_ids=sample['input_ids'], attention_mask=sample['attention_mask'], sentence_ids=sample['sentence_ids'])

In [105]:
start_logits, end_logits = output

In [110]:
start, end = start_logits.argmax(), end_logits.argmax()

In [115]:
from transformers import AutoTokenizer

In [117]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base-expanded-embeddings')

In [119]:
tokenizer.decode(sample['input_ids'][:, start:end][0])

' David W. Eaton'

In [112]:
start

tensor(212)

In [113]:
end

tensor(216)